In [333]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import gregw

In [261]:
train = pd.read_csv("house_prices_train.csv")
print len(train)
test = pd.read_csv("housing_prices_test.csv")

1460


In [182]:
y = train['SalePrice']
train.drop(['SalePrice'], axis = 1, inplace = True)

In [183]:
test_id = test['Id']

In [254]:
alldata = pd.concat([train, test], axis=0)
alldata.drop(['Id'], axis = 1, inplace = True)

In [257]:
alldata['MSSubClass'] = alldata['MSSubClass'].astype(object)
alldata['MoSold'] = alldata['MoSold'].astype(object)
alldata['YrSold'] = alldata['YrSold'].astype(object)
alldata['BsmtFullBath'] = alldata['BsmtFullBath'].astype(object)
alldata['BsmtHalfBath'] = alldata['BsmtHalfBath'].astype(object)
alldata.LotFrontage = alldata.LotFrontage.fillna(0)
alldata.Alley = alldata.Alley.fillna('None')
alldata.FireplaceQu = alldata.FireplaceQu.fillna('None')
alldata.GarageType = alldata.GarageType.fillna('None')
alldata['GarageYrBlt'] = alldata['GarageYrBlt'].fillna('None')
alldata['GarageFinish'] = alldata['GarageFinish'].fillna('None')
alldata['GarageCars'] = alldata['GarageCars'].fillna(0)
alldata['GarageArea'] = alldata['GarageArea'].fillna(519)
alldata['GarageQual'] = alldata['GarageQual'].fillna('None')
alldata['GarageCond'] = alldata['GarageCond'].fillna('None')
alldata['PoolQC'] = alldata['PoolQC'].fillna('None')
alldata['Fence'] = alldata['Fence'].fillna('None')
alldata['MiscFeature'] = alldata['MiscFeature'].fillna('None')

In [258]:
gregw.impute_cal('MSZoning', alldata)
gregw.impute_cal('Utilities', alldata)
gregw.impute_cal('Exterior1st', alldata)
gregw.impute_cal('Exterior2nd', alldata)
gregw.impute_cal('MasVnrType', alldata)
gregw.impute_reg('MasVnrArea', alldata)
gregw.impute_cal('BsmtQual', alldata)
gregw.impute_cal('BsmtCond', alldata)
gregw.impute_cal('BsmtExposure', alldata)
gregw.impute_cal('BsmtFinType1', alldata)
gregw.impute_reg('BsmtFinSF1', alldata)
gregw.impute_cal('BsmtFinType2', alldata)
gregw.impute_cal('Electrical', alldata)
gregw.impute_cal('BsmtFullBath', alldata)
gregw.impute_cal('BsmtHalfBath', alldata)
gregw.impute_cal('KitchenQual', alldata)
gregw.impute_cal('Functional', alldata)
gregw.impute_reg('GarageCars', alldata)
gregw.impute_cal('SaleType', alldata)

Empty DataFrame
Columns: [MSZoning, LotFrontage, LotArea, OverallQual, OverallCond, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, 1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea, FullBath, HalfBath, BedroomAbvGr, KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars, GarageArea, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, MSSubClass_30, MSSubClass_40, MSSubClass_45, MSSubClass_50, MSSubClass_60, MSSubClass_70, MSSubClass_75, MSSubClass_80, MSSubClass_85, MSSubClass_90, MSSubClass_120, MSSubClass_150, MSSubClass_160, MSSubClass_180, MSSubClass_190, MSZoning_FV, MSZoning_RH, MSZoning_RL, MSZoning_RM, Street_Pave, Alley_None, Alley_Pave, LotShape_IR2, LotShape_IR3, LotShape_Reg, LandContour_HLS, LandContour_Low, LandContour_Lvl, Utilities_NoSeWa, LotConfig_CulDSac, LotConfig_FR2, LotConfig_FR3, LotConfig_Inside, LandSlope_Mod, LandSlope_Sev, Neighborhood_Blueste, Neighborhood_BrDale, Neighborhood_BrkSide, Neighborhood_

In [259]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 79 columns):
MSSubClass       2919 non-null object
MSZoning         2919 non-null object
LotFrontage      2919 non-null float64
LotArea          2919 non-null int64
Street           2919 non-null object
Alley            2919 non-null object
LotShape         2919 non-null object
LandContour      2919 non-null object
Utilities        2919 non-null object
LotConfig        2919 non-null object
LandSlope        2919 non-null object
Neighborhood     2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
BldgType         2919 non-null object
HouseStyle       2919 non-null object
OverallQual      2919 non-null int64
OverallCond      2919 non-null int64
YearBuilt        2919 non-null int64
YearRemodAdd     2919 non-null int64
RoofStyle        2919 non-null object
RoofMatl         2919 non-null object
Exterior1st      2919 non-null object
Exterior2nd      2919 n

In [270]:
dummies_data = pd.get_dummies(alldata,drop_first=True)

In [351]:
train = dummies_data.iloc[:1460]
test = dummies_data.iloc[1460:]

In [357]:
cross_val_score(XGBRegressor(max_depth=3, n_estimators=115), train, y, cv = 5).mean()

0.88128228702215683

In [287]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=3)

In [348]:
parameters = {'max_depth':(2,3), 'n_estimators':(110,115,120)}
XGBR = XGBRegressor()
XGBR = GridSearchCV(XGBR, parameters)


XGBR.fit(X_train, y_train)


GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': (110, 115, 120), 'max_depth': (2, 3)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [349]:
print XGBR.best_params_
XGBR.score(X_test, y_test)

{'n_estimators': 120, 'max_depth': 2}


0.893211538789635

In [317]:
ans['SalePrice'] = pd.DataFrame(XGBR.predict(test))

In [318]:
final = pd.concat([test_id, ans['SalePrice']], axis=1)

In [328]:
final.to_csv('kaggle_house_prices.csv', index=False)

,Id,SalePrice
0,1461,123276.117188
1,1462,158738.312500
2,1463,172211.437500
3,1464,182366.109375
4,1465,195514.812500
5,1466,171767.078125
6,1467,167138.140625
7,1468,166837.546875
8,1469,195058.531250
9,1470,126682.195312
